# Installing required packages

In [ ]:
!pip install nemo_toolkit[all]
!pip install pydub
!pip install jsonlines
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import os
import librosa
import nemo
import nemo.collections.asr as nemo_asr
from tqdm import tqdm
import jsonlines
from jiwer import wer, cer
import pytorch_lightning as pl
import jsonlines
from sklearn.model_selection import train_test_split
from omegaconf import DictConfig
from google.colab import drive
import gc
import torch
import warnings
drive.mount('/content/drive')

[NeMo W 2023-05-23 15:31:21 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-23 15:31:24 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


# Define the model

In [ ]:
asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_ru_conformer_transducer_large")

[NeMo I 2023-05-23 15:31:25 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_ru_conformer_transducer_large/versions/1.13.0/files/stt_ru_conformer_transducer_large.nemo to /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo
[NeMo I 2023-05-23 15:31:55 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-05-23 15:31:57 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-23 15:31:58 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-05-23 15:31:58 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 16
    shuffle: fals

[NeMo I 2023-05-23 15:31:58 features:287] PADDING: 0


[NeMo W 2023-05-23 15:32:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-05-23 15:32:02 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-23 15:32:05 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo.


# Download the manifest

Manifest contains path to audio, transcription and duration of the audio.

In [ ]:
manifest = []
with jsonlines.open('/content/drive/MyDrive/manifest.jsonl') as f:
    for obj in f:
        manifest.append(obj)

# Test the model without fine-tuning

In [ ]:
files = [file['audio_filepath'] for file in manifest]
transcriptions = asr_model.transcribe(paths2audio_files=files)

In [ ]:
wers = []
cers = []

for i, transcription in enumerate(transcriptions[0]):
    if manifest[i]['text'] != '' and manifest[i]['text'] != ' ':
      w = wer(manifest[i]['text'], transcription)
      wers.append(w)
      c = cer(manifest[i]['text'], transcription)
      cers.append(c)
      manifest[i]['transcript'] = transcription

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.49119162082399914
Mean CER:  0.30814035001674916


# Train

## Define trainer from pytorch lightning

In [ ]:
trainer = pl.Trainer(max_epochs=5, accelerator="cuda")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


## Split data to train and test samples

In [ ]:
train, test = train_test_split(manifest, test_size=0.3, random_state=22)

with jsonlines.open('train.jsonl', 'w') as writer:
    writer.write_all(train)
with jsonlines.open('test.jsonl', 'w') as writer:
    writer.write_all(test)

## Change parameters in the config

In [ ]:
max_dur = 0
for m in manifest:
    if m['duration'] > max_dur:
        max_dur = m['duration']

params = asr_model._cfg
params['optim']['lr'] = 0.001
params['train_ds']['manifest_filepath'] = '/content/train.jsonl'
params['train_ds']['batch_size']=2
params['validation_ds']['manifest_filepath'] = '/content/test.jsonl'
params['validation_ds']['batch_size']=2
params['test_ds']['manifest_filepath'] = '/content/test.jsonl'
params['test_ds']['batch_size']=2
params['train_ds']['num_workers'] = 0
params['train_ds']['max_duration'] = max_dur
params['train_ds']['min_duration'] = 0.01

In [ ]:
cfg_ = DictConfig(params)
cfg_.joint.fuse_loss_wer = True
cfg_.joint.fused_batch_size = 2
cfg_.joint.preserve_memory = True
cfg_.log_prediction = False
cfg_.optim.sched.warmup_steps = None

## Restore model with our new config

In [ ]:
asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_ru_conformer_transducer_large", override_config_path=cfg_)
asr_model.setup_training_data(cfg_['train_ds'])
asr_model.setup_multiple_validation_data(cfg_['validation_ds'])

[NeMo I 2023-05-23 15:32:44 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo.
[NeMo I 2023-05-23 15:32:44 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo
[NeMo I 2023-05-23 15:32:44 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-05-23 15:32:48 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-23 15:32:50 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    batch_size: 2
    shuffle: true
    num_workers: 0
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 37.02
    min_duration: 0.01
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-05-23 15:32:50 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /content/test.jsonl
    sample_rate: 16000
    

[NeMo I 2023-05-23 15:32:50 features:287] PADDING: 0


[NeMo W 2023-05-23 15:32:54 rnnt:1211] `preserve_memory` was set for the Joint Model. Please be aware this will severely impact the forward-backward step time. It also might not solve OOM issues if the GPU simply does not have enough memory to compute the joint.


[NeMo I 2023-05-23 15:32:55 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-23 15:32:56 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.18.1/stt_ru_conformer_transducer_large/15cac96975e1690f6ef8c9ab6deae578/stt_ru_conformer_transducer_large.nemo.
[NeMo I 2023-05-23 15:32:57 collections:193] Dataset loaded with 5545 files totalling 4.65 hours
[NeMo I 2023-05-23 15:32:57 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-23 15:32:59 collections:193] Dataset loaded with 2377 files totalling 2.02 hours
[NeMo I 2023-05-23 15:32:59 collections:194] 0 files were filtered totalling 0.00 hours


[NeMo W 2023-05-23 15:32:59 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


In [ ]:
gc.collect()

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

if accelerator == 'gpu':
  torch.cuda.empty_cache()

In [ ]:
warnings.filterwarnings('ignore')

## Train model

In [ ]:
trainer.fit(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-05-23 15:38:57 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2023-05-23 15:38:57 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f478925c7c0>" 
    will be used during training (effective maximum steps = 13865) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 13865
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 115 M 
2 | decoder           | RNNTDecoder                       | 3.9 M 
3 | joint             | RNNTJoint                         | 1.4 M 
4 | loss              | RNNTLoss                          | 0     
5 | spec_augmentation | SpectrogramAugmentation           | 0     
6 | wer               | RNNTBPEWER                        | 0     
------------------------------------------------------------------------
120 M     Trainable params
0         Non-trainable params
120 M     Total params
481.780   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


## Save model to disk

In [ ]:
asr_model.save_to('/content/drive/MyDrive/finetuned_conformer.nemo')

In [ ]:
model = nemo_asr.models.EncDecRNNTBPEModel.restore_from(restore_path='/content/drive/MyDrive/finetuned_conformer.nemo')

[NeMo I 2023-05-23 18:09:40 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-23 18:09:41 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/train.jsonl
    sample_rate: 16000
    batch_size: 2
    shuffle: true
    num_workers: 0
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 37.02
    min_duration: 0.01
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    bucketing_weights: ''
    
[NeMo W 2023-05-23 18:09:41 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /content/test.jsonl
    sample_rate: 16000
    

[NeMo I 2023-05-23 18:09:41 features:287] PADDING: 0


[NeMo W 2023-05-23 18:09:44 rnnt:1211] `preserve_memory` was set for the Joint Model. Please be aware this will severely impact the forward-backward step time. It also might not solve OOM issues if the GPU simply does not have enough memory to compute the joint.


[NeMo I 2023-05-23 18:09:44 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-23 18:09:44 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /content/drive/MyDrive/finetuned_conformer.nemo.


# Evaluate

In [ ]:
files = [file['audio_filepath'] for file in test]
transcriptions = model.transcribe(paths2audio_files=files)

In [ ]:
wers = []
cers = []

for i, transcription in enumerate(transcriptions[0]):
    if test[i]['text'] != '' and test[i]['text'] != ' ':
      w = wer(test[i]['text'], transcription)
      wers.append(w)
      c = cer(test[i]['text'], transcription)
      cers.append(c)
      test[i]['wer'] = w
      test[i]['cer'] = c
      test[i]['transcript'] = transcription

print('Mean WER: ', sum(wers)/len(wers))
print('Mean CER: ', sum(cers)/len(cers))

Mean WER:  0.352470673234719
Mean CER:  0.21391450961593267


In [ ]:
test_df = pd.DataFrame.from_records(test)

In [ ]:
test_df.to_excel("output_conformer.xlsx")